In [1]:
import pandas  as pd
import numpy as np
import requests
import json

In [2]:
url = "https://research.ibm.com/graphql"
payload = json.dumps({
  "operationName": "BlogPostsQuery",
  "variables": {
    "start": 0,
    "limit": 13700,
    "filterBy": {
      "tag": "artificial-intelligence"
    }
  },
  "query": "query BlogPostsQuery($start: Int!, $limit: Int!, $filterBy: BlogPostsFilterBy, $ignoreIDs: [ID]) {\n  blogPostsSearch(\n    start: $start\n    limit: $limit\n    filterBy: $filterBy\n    ignoreIDs: $ignoreIDs\n  ) {\n    totalCount\n    nodes {\n      ...LandingBlogPost\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment LandingBlogPost on BlogPost {\n  id\n  slug\n  title\n  category: blog_category {\n    id\n    name\n    slug\n    __typename\n  }\n  cover_image {\n    id\n    url\n    alternativeText\n    width\n    height\n    __typename\n  }\n  published: publish_at\n  read_duration\n  tags {\n    id\n    name\n    slug\n    __typename\n  }\n  authors: blog_authors {\n    id\n    name\n    __typename\n  }\n  __typename\n}\n"
})
headers = {
  'Cookie': 'CISESSIONIDPR02A02=PBC5YS:2551732044; BMAID=57f0f1a8-fee4-4768-8dd5-43d411a3c0d8; com.ibm.cloud.iam.LoggedIn.prod=1; _ga=GA1.2.419780969.1632356809; AMCVS_D10F27705ED7F5130A495C99%40AdobeOrg=1; AMCV_D10F27705ED7F5130A495C99%40AdobeOrg=359503849%7CMCMID%7C57033701665934880152906176239228574186%7CMCAAMLH-1632961609%7C4%7CMCAAMB-1632961609%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1632364009s%7CNONE%7CvVersion%7C5.0.1; OPTOUTMULTI=0:0%7Cc1:1%7Cc2:0%7Cc3:0; utag_main=v_id:017c100c3ae8003fb03a8c51dffc03072002306a0086e$_sn:1$_se:6$_ss:0$_st:1632358621617$ses_id:1632356809451%3Bexp-session$_pn:1%3Bexp-session$is_country_requiring_explicit_consent:true; CISESSIONIDPR02B02=PBC5YS:839392655; notice_gdpr_prefs=0|1|2:; notice_preferences=2:; cmapi_cookie_privacy=permit_1|2|3; ajs_user_id=%22IBMid-6620027VLR%22; ajs_anonymous_id=%22485204a0-a6f3-4c85-be66-e40d8fa4a085%22; userContext=57f0f1a8-fee4-4768-8dd5-43d411a3c0d8|0|0|0|BR|SP|1|2:|implied; notice_behavior=implied|eu; cmapi_gtm_bl=; 745fa666c78bd1f7f49187a9d5f427e0=c6301843c9cd55bef194bfee503c4a96; fd2684f1b7a13a2f01659fff67dbe3d6=634d0123f007d8ac1896823c5f885ebb; pageviewContext=601e6ae2-0663-447e-a848-b6f4718543f2; _abck=923FE248721EB9914E3E4A1BCF433215~0~YAAQXN4AyaHtmMp8AQAAhj+m4QYdEet+uPaypK1KfoTHgbRlP0Put3wTxPgmWNJDIGH93wtu017fmUsvio8kWnWF/J7RHdxvcMD+t+EB3FPL39NMLuqAZGdWGd76rnnvezTBPv4LfoXMPLmUg6oTOFUfPzcCepPJjXhM80SWFT4+jQaHrYjaG1tmNkWnIPd3PaTXKnV6cwFDcE7suWaGeANpwXNim12epjtVpBqE6hj53aSTHCGwYamOyCotHuEo3NlS03HRA2wu1QrT7weRCpX7Gum4Zh7c+sxa70HNykJwt5HG3GKNHFIBC9KIbWEUR6lepg+TLXLXcXOI+vI0hdnxfEvfBq8Q0xdiSnX7AY8tiiN7G9TkN01xDR/ulOPGVQlkosN/YEChRAsYVagHPLDthCXR~-1~-1~-1; ak_bmsc=8D979BE4A9C9D47420DDE9165E4BB11B~000000000000000000000000000000~YAAQXN4AyaLtmMp8AQAAhj+m4Q35oI4ZRL5CfxcYmxghkNLhU6/e0SQDWCa5rFKnfqCAbO1zKxYMyvE+0COqXXRMN7axObpyuhN2AfyJrOWwctRnfqwNCmsoZAXbeK5ErUFmyzXt4VN4Yr77orwmbY6PAxO9PAGCse8nT0WyQ9RioapNxbtxgoeorV9EJSBUgo26P1CCd/Xf8ikZ7OsCQEgeN6fbdiGjsuyLmbFp6IPckSzfMkVIRNiDNXA8LiKtNiJHHH77BO9uiiiBnjsEh2bjzV3x4w95TWEBiNwb6hPzM5DtUfmYb6OENfzE/ix09Eo0J3aUdGNLPV0ZO9qEwqTlaK2sYHiCtJwBgM/RfIaA9f9wpHSEkuKJTWjnaveYZaiQp35c; bm_sz=3B92BA7BF3307A68DFA57DF7C6E739CD~YAAQXN4AyaPtmMp8AQAAhj+m4Q13c8ciU2k25xbWGhCSRDnfuiqY2Tn16JXAU5hgxE02HULbQVVPxP8GeXOJQtcI/9lhgmwsk3nfBRthOIi/Tv/EX1JAwiW2SQZ2TncmDYsUbZ/fMDEVYElBN/QmkwAkuPs7XGU4tsvePvzA1makvz88Q6SXsNa8H24X/QyjbRQZLL20G2QwGSC6iTaNqdlFOYe4TJQZ6aC7y6lEWAVYm/Ew3CUHzjB2eyVIPB/j/OQ7JgVRLEIkQMVeXrG8Ix/sO7HmyXhfZ+p8re8U2Pg=~3551811~3359029; 745fa666c78bd1f7f49187a9d5f427e0=80027c1f5a26dabe076e2f212367a56c; fd2684f1b7a13a2f01659fff67dbe3d6=52c4d32b293d9b0f00366b34772dfc8d',
  'Content-Type': 'application/json'
}

In [3]:
response = requests.request("POST", url, headers=headers, data=payload)

json_f = response.json()

In [4]:
dic = json.dumps(json_f)
real = json.loads(dic)
df = pd.DataFrame(real)
df

,data
blogPostsSearch,"{'totalCount': 137, 'nodes': [{'id': '422', 's..."


In [174]:
df1 = pd.DataFrame(df['data'].values.tolist()[0])
titles = pd.DataFrame(df1.nodes.values.tolist()).iloc[:,]
author1 = pd.DataFrame(titles.authors.values.tolist()).iloc[:,]
titles2 = titles.category.replace(to_replace='None', value=np.nan).dropna()
#author1.iloc[:,0].values.tolist()
category = pd.DataFrame(titles2.values.tolist()).rename({'name': 'Source'}, axis=1).drop(['id', 'slug', '__typename'], axis=1)
tags = pd.DataFrame(pd.DataFrame(titles.tags.values.tolist()).iloc[:,0].values.tolist()).rename({'name': 'Tag'}, axis=1).drop(['id', 'slug', '__typename'], axis=1)
authors = pd.DataFrame(pd.DataFrame(titles.authors.values.tolist()).iloc[:,0].replace('None', np.nan).dropna().values.tolist()).drop(['id', '__typename'], axis=1)

In [302]:
titles2 = titles.drop(labels=['category', 'cover_image', 'tags', 'authors'], axis=1)
df2 = pd.concat([titles2, category, tags, authors], axis=1)
df3 = df2.drop(['id', '__typename', 'slug'], axis=1).rename({'name': 'Contributor', 'title': 'Title', 'published': 'Publised', 'read_duration': 'Read Duration'}, axis=1)
df3.Publised = pd.to_datetime(df3.Publised).dt.date
df3.rename({'Publised': 'Date Published'}, axis=1, inplace=True)
# dff.Source.value_counts()
df4=    df3.pivot_table(index=['Tag', 'Source', 'Date Published', 'Contributor'],
                    values=['Title'],
                    aggfunc=lambda x: ' '.join(x))

pd.options.display.max_colwidth = 200

In [306]:
pd.set_option('display.colheader_justify', 'center')
df4.to_csv('ibm_ai.csv')